In [186]:
#必要运行库，saucenao_api代码需修改
#pip install -U saucenao_api
#pip install --upgrade urllib3
import os
from PIL import Image
import shutil
from saucenao_api import SauceNao
import time
#gelbooru爬虫使用库
import requests
from bs4 import BeautifulSoup
import json


#gelbooru_headers
gelbooru_headers = {
     ''
     }

pixiv_headers = {
     'Cookie':'',
     'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36 Edg/124.0.0.0'
     }


#封装函数
#gelbooru下载
def gelbooru_download_picture(download_url,org_pic_url,picture_name,save_directory):#下载网页链接用于提取网页id打印结果，提取出的原图链接用于下载，图片名字，保存目录
    #使用requests爬取
    response = requests.get(org_pic_url,stream=True,headers=gelbooru_headers,timeout=20)#单线程，分块下载，超时时间20秒
    if response.status_code == 200:
            #文件名为默认图片名
            file_save = os.path.join(save_directory, picture_name)
            # 将响应内容写入文件
            with open(file_save, 'wb') as f:
                f.write(response.content)
            print(f'gelbooru的id：{download_url.split('=')[3]}，保存的文件名：{picture_name}，下载完成')
    else:
            print(f'无法下载网页{download_url.split('=')[3]}状态码：{response.status_code}')

#gelbooru获取原图链接并下载
def gelbooru_download(gelbooru_id,save_directory):
    download_url = 'https://gelbooru.com/index.php?page=post&s=view&id='+ str(gelbooru_id)
    website_text = requests.get(download_url,headers = gelbooru_headers).text#发送get请求并将内容转化成文本
    soup = BeautifulSoup(website_text,features='lxml')#lxml解析网页内容
    soup.find_all(id='image')[0]['src']#找到含有id='image'内容的
    picture_site = soup.find_all(id='image')[0]['src'].split('/')#用/切分sample网站
    if picture_site[4] == 'images':
        org_pic_url = '/'.join(picture_site)#原图网址
        gelbooru_download_picture(download_url,org_pic_url,picture_site[7],save_directory)
    if picture_site[4] == 'samples':
        picture_site[7] = picture_site[7].split('_')[1]#修改切分后内容，改为原图网址
        picture_site[4] = 'images'
        org_pic_url = '/'.join(picture_site)#原图网址
        gelbooru_download_picture(download_url,org_pic_url,picture_site[7],save_directory)

#gelbooru获取Source网址
def get_gelbooru_source(gelbooru_id):
    download_url = 'https://gelbooru.com/index.php?page=post&s=view&id='+ str(gelbooru_id)
    website_text = requests.get(download_url,headers = gelbooru_headers).text#发送get请求并将内容转化成文本
    soup = BeautifulSoup(website_text,features='lxml')#lxml解析网页内容
    source_site = soup.find_all('a',rel="nofollow")[0]['href']#获取到pixiv的id
    return source_site

#从gelbooru获取的Source网址中提取出对应推特网址或者pixiv的id
def get_gelbooru_source_id(source_site):
    pixiv_id = '非pixiv源'
    twitter_website = '非twitter源'
    if 'www.pixiv.net' in source_site.split('/'):
        pixiv_id = source_site.split('/')[4]#pixiv的id
    if 'twitter.com' in source_site.split('/'):
        twitter_website = source_site#推特记录网址
    return pixiv_id,twitter_website

#提取搜索结果
def get_ulr_id(url_pool):
    pixiv_results=[]
    gelbooru_results=[]
    twitter_results=[]
    for url in url_pool:#对池中url依次判断是否为需要的网站
        if 'pixiv' in url.split('.'):
            if url.split('=')[2] not in pixiv_results:#结果中可能有多个p站链接
                pixiv_results.append(url.split('=')[2])
        if 'twitter.com' in url.split('/'):
            if url not in twitter_results:#结果中可能有多个链接
                twitter_website.append(url)
        if 'gelbooru.com' in url.split('/'):
            if url.split('=')[3] not in gelbooru_results:#结果中可能有多个链接
                gelbooru_results.append(url.split('=')[3])
            pixiv_id,twitter_website = get_gelbooru_source_id(get_gelbooru_source(url.split('=')[3]))#先通过get_gelbooru_source函数得到来源，再用get_gelbooru_source_id得到id
            if pixiv_id not in pixiv_results and pixiv_id != '非pixiv源':#get_gelbooru_source_id函数默认返回
                pixiv_results.append(pixiv_id)#将得到的p站id再次加入pixiv_results
            if twitter_website not in twitter_results and twitter_website != '非twitter源':#get_gelbooru_source_id函数默认返回
                twitter_results.append(twitter_website)#将得到的推特网址再次加入twitter_results
    return pixiv_results,gelbooru_results,twitter_results

def SauceNao_Search(picture_path,sauce_api):#待搜索图片路径，搜索api
    #最终结果存储数组
    pixiv_id_results=[]
    gelbooru_id_results=[]
    twitter_website_results=[]
    search_picture_list=[]
    now_search_picture = ' '#存储当前搜索图片名称
    #os.listdir()方法获取文件夹名字，返回数组
    count=0#搜索次数
    file_name_list = os.scandir(picture_path)
    for file in file_name_list:
        if file.is_dir() == False:
            search_picture_list.append(file)#添加搜索完成的图片名
            now_search_picture = file.name#保存当前搜索的文件名
            url_pool = []#将本轮results搜索到的链接全部加入其中，放在循环中，每轮清空
            sauce_search_results = SauceNao(sauce_api).from_file(picture_path + '\\' + file.name)  # or from_url()
            time.sleep(10)#每次搜索之间间隔
            for source in sauce_search_results:#提取档次搜索结果所有网址到url_pool
                if source.similarity > 65:#判断相似度大于x的搜索结果，并将结果加入url_pool
                        url_pool.extend(source.urls)
            pixiv_id_results.extend(get_ulr_id(url_pool)[0])
            gelbooru_id_results.extend(get_ulr_id(url_pool)[1])
            twitter_website_results.extend(get_ulr_id(url_pool)[2])
            count=count+1
            print(f'{count}图完成：{now_search_picture}，pixiv_id：{get_ulr_id(url_pool)[0]}，gelbooru_id：{get_ulr_id(url_pool)[1]}，推特：{get_ulr_id(url_pool)[2]}')
    return search_picture_list,pixiv_id_results,gelbooru_id_results,twitter_website_results,now_search_picture
    #返回结果依次为已搜索图片，pixiv搜索到的id，gelbooru搜索到的id，pixiv网址，gelbooru网址，当前搜索的图片名称

#p站关注爬取函数
def get_pixiv_following(id,pixiv_headers):
    #默认参数，不需要改动；从offset开始，得到limit的个数，所以offset可以为任意值，limit为100，单次得到offest+limit范围内的关注
    limit = 100
    rest = 'show'
    tag = ''
    lang = 'zh'
    user_ids = []#保存关注id
    following_numbers = json.loads(requests.get(f'https://www.pixiv.net/ajax/user/extra?lang={lang}',headers = pixiv_headers).text)['body']['following']#获取关注数量
    for offset in range(following_numbers//100 + 1):
        following_response = requests.get(f'https://www.pixiv.net/ajax/user/{id}/following?offset={offset*100}&limit={limit}&rest={rest}&tag={tag}&acceptingRequests=0&lang={lang}',headers = pixiv_headers).text#单次循环得到100个关注
        time.sleep(1)#每次之间间隔，防止请求太频繁
        following_json_data = json.loads(following_response)#json库解析内容
        for user in following_json_data["body"]["users"]:
            user_ids.append(user["userId"])
    return user_ids

#得到单张p站图的作者id
def get_pixiv_author(picture_id):
    picture_authorid = json.loads(requests.get(f'https://www.pixiv.net/ajax/illust/{picture_id}',headers = pixiv_headers).text)['body']['tags']['authorId']
    return picture_authorid

#系统级代理
os.environ["http_proxy"] = 'http://127.0.0.1:7890'
os.environ["https_proxy"] = 'http://127.0.0.1:7890'
Image.MAX_IMAGE_PIXELS = None# 关闭DecompressionBombWarning警告


In [ ]:
_,pixiv_id_results,gelbooru_id_results,twitter_website_results,now_search_picture = SauceNao_Search(r'F:\Picture\Search\now','')


In [ ]:
pixiv_id_results

In [ ]:
for id in pixiv_id_results: print(id)#打印p站id

In [ ]:
#gelbooru爬取上面得到的图片id
gid=0
for id in gelbooru_id_results:
    if id != gelbooru_id_results[0] or id != gelbooru_id_results[-1]:#非首尾文件暂停时间
        time.sleep(8)#每次间隔
        gelbooru_download(id,r'C:\Users\13213\Downloads\gelbooru')
    gid=gid+1

In [ ]:
#当前搜索文件名
now_search_picture.name

                                                            调试区

In [139]:
url_pool=[]
for source in sauce_search_results:#提取档次搜索结果所有网址到url_pool
    if source.similarity > 65:#判断相似度大于x的搜索结果，并将结果加入url_pool
        url_pool.extend(source.urls)

In [ ]:
url_pool

In [151]:
def get_ulr_id(url_pool):
    pixiv_results=[]
    gelbooru_results=[]
    twitter_results=[]
    c=0
    for url in url_pool:#对池中url依次判断是否为需要的网站
        if 'pixiv' in url.split('.'):
            if url.split('=')[2] not in pixiv_results:
            print(c,pixiv_results)
            pixiv_results.append(url.split('=')[2])
            print(c,pixiv_results)
        if 'twitter.com' in url.split('/'):
            twitter_website.append(url)
        if 'gelbooru.com' in url.split('/'):
            gelbooru_results.append(url.split('=')[3])
            pixiv_id,twitter_website = get_gelbooru_source_id(get_gelbooru_source(url.split('=')[3]))#先通过get_gelbooru_source函数得到来源，再用get_gelbooru_source_id得到id
            print(c,'if前',pixiv_results)
            if pixiv_id not in pixiv_results and pixiv_id != '非pixiv源':#get_gelbooru_source_id函数默认返回
                pixiv_results.append(pixiv_id)#将得到的p站id再次加入pixiv_results
            print(c,'if后',pixiv_results)
            if twitter_website not in twitter_results and twitter_website != '非twitter源':#get_gelbooru_source_id函数默认返回
                twitter_results.append(twitter_website)#将得到的推特网址再次加入twitter_results
        c=c+1
    return pixiv_results,gelbooru_results,twitter_results

In [160]:
def abc():
    return 1,2,3

In [167]:
a = abc()[0]
a

1